In [13]:
import os
import requests
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-1106")

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )

    

class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


class CompanyOverviewArgsSchema(BaseModel):
    symbol : str = Field(description="Stock symbol of the company. Example: AAPL, TSLA")

class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description="""
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()


class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyIncomeStatement"
    description="""
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()["annualReports"]
    

class CompanyStockPerformanceTool(BaseTool):
    name = "CompanyStockPerformance"
    description="""
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()
    
agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool()
    ],
)


prompt = "Give me financial information on CRISPR Therapeutics A's stock, considering its financials, income statements and stock performance and help me analyze if it's a potential good investment."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Stock Market Symbol for CRISPR Therapeutics A'}`


The latest CRISPR Therapeutics stock prices, stock quotes, news, and CRSP history to help you invest and trade smarter. ... CRISPR Therapeutics AG's market capitalization is $6.85 B by 83.35 M ... Stock Quote & Chart. (Common Stock) 3:22 PM EST on Feb 12, 2024. Change. Volume. Today's Open. Previous Close. See the latest CRISPR Therapeutics AG stock price (CRSP:XNAS), related news, valuation, dividends and more to help you make your investing decisions. Vertex/CRISPR price sickle cell disease gene therapy at $2.2 mln. Vertex Pharmaceuticals and its partner CRISPR Therapeutics said on Friday their sickle cell disease gene therapy Casgevy would be available at a list price of $2.2 million in the United States. Other symbols: VRTX. 2 months ago - Reuters. CRISPR Therapeutics AG (CRSP.NASDAQ): Stock quote, stock chart, quotes, analysis, advice

{'input': "Give me financial information on CRISPR Therapeutics A's stock, considering its financials, income statements and stock performance and help me analyze if it's a potential good investment.",
 'output': 'Here\'s the financial information for CRISPR Therapeutics AG (CRSP):\n\n1. Company Overview:\n   - Name: Crispr Therapeutics AG\n   - Description: CRISPR Therapeutics AG is a gene editing company that focuses on the development of gene-based transformative drugs for serious human diseases. The company is headquartered in Zug, Switzerland.\n   - Market Capitalization: $7.4 billion\n   - Sector: Life Sciences\n   - Industry: Biological Products\n   - Revenue (TTM): $371.2 million\n   - Gross Profit (TTM): -$570.7 million\n   - Earnings Per Share (EPS): -$1.94\n   - Price/Earnings Ratio (P/E): Not available\n   - Analyst Target Price: $84.58\n   - Analyst Ratings: 6 Strong Buy, 8 Buy, 11 Hold, 3 Sell\n\n2. Income Statement (2023):\n   - Total Revenue: $371.3 million\n   - Gross 